## Prerequisites



In [0]:
import pandas as pd 
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

### Note! Some of these models support only multiclass classification, please, while selecting your dataset,  
### be sure that for algorithms which does not support multilabel classification you use only examples with only one label. 
### Examples without a label in any of the provided categories are clean messages, without any toxicity.

In [114]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/My Drive/train.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [115]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [116]:
df.shape

(159571, 8)

### As one of the methods to make the training simpier, use only examples, assigned to any category vs clean examples.  
For example:  
- Select only messages with obscene label == 1  
- Select all of the "clean" messages  
Implement a model which can perform a binary classification  - to understand whether your message is obscene or not.   

##### If you want to perform a multilabel classification, please understand the difference between multilabel and multiclass classification and be sure that you are solving the correct task - choose only algorithms applicable for solving this type of problem.

#### To work with multiclass task:  
You only need to select messages which have only one label assigned: message cannot be assigned to 2 or more categories.  

#### To work with multilabel task: 
You can work with the whole dataset - some of your messages have only 1 label, some more than 1. 

## Text vectorization

Previously we worked only with words vectorization. But we need to have a vector for each text, not only words from it. 

Before starting a text vectorization, please, make sure you are working with clean data - use the dataset created on the previous day. Cleaned from punctuation, stop words, lemmatized or stemmed, etc. 

In [117]:
from string import punctuation

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import nltk

nltk.download('stopwords')
  
lemmatizer = WordNetLemmatizer() 
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [118]:
import nltk

nltk.download('punkt')
nltk.download('wordnet')

def preprocess_text(tokenizer, lemmatizer, stop_words, punctuation, text): 
    tokens = tokenizer(text.lower())
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return [token for token in lemmas if token not in stop_words and token not in punctuation]

df['cleaned'] = df.comment_text.apply(lambda x: preprocess_text(word_tokenize, lemmatizer, stop_words, punctuation, x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [119]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,..."
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[d'aww, match, background, colour, 'm, seeming..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, 'm, really, trying, edit, war, 's, ..."
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0,"[``, ca, n't, make, real, suggestion, improvem..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page, 's]"


In [0]:
def flat_nested(nested):
    flatten = []
    for item in nested:
        if isinstance(item, list):
            flatten.extend(item)
        else:
            flatten.append(item)
    return flatten

In [0]:
vocab = set(flat_nested(df.cleaned.tolist()))

In [122]:
len(vocab)

249736

As we see, probably you vocabulary is too large.  
Let's try to make it smaller.  
For example, let's get rig of words, which has counts in our dataset less than some threshold.

In [0]:
from collections import Counter, defaultdict 

cnt_vocab = Counter(flat_nested(df.cleaned.tolist()))

In [124]:
cnt_vocab.most_common(10)

[("''", 242528),
 ('``', 155370),
 ('article', 73284),
 ("'s", 66767),
 ("n't", 57144),
 ('wa', 56592),
 ('page', 56263),
 ('wikipedia', 45418),
 ('talk', 35356),
 ('ha', 31896)]

You can clean words which are shorter that particular length and occur less than N times. 

In [0]:
threshold_count = 10
threshold_len = 4 
cleaned_vocab = [token for token, count in cnt_vocab.items() if count > threshold_count and len(token) > threshold_len]

In [126]:
len(cleaned_vocab)

18696

Much better!  
Let's try to vectorize the text summing one-hot vectors for each word. 

In [0]:
vocabulary = defaultdict()

for i, token in enumerate(cleaned_vocab): 
    empty_vec = np.zeros(len(cleaned_vocab))
    empty_vec[i] = 1 
    vocabulary[token] = empty_vec

In [128]:
vocabulary['source']

array([0., 0., 0., ..., 0., 0., 0.])

Rigth now we have vectors for words (words are one-hot vectorized)  
Let's try to create vectors for texts: 

In [129]:
sample_text = df.cleaned[10]
print(sample_text)

['``', 'fair', 'use', 'rationale', 'image', 'wonju.jpg', 'thanks', 'uploading', 'image', 'wonju.jpg', 'notice', 'image', 'page', 'specifies', 'image', 'used', 'fair', 'use', 'explanation', 'rationale', 'use', 'wikipedia', 'article', 'constitutes', 'fair', 'use', 'addition', 'boilerplate', 'fair', 'use', 'template', 'must', 'also', 'write', 'image', 'description', 'page', 'specific', 'explanation', 'rationale', 'using', 'image', 'article', 'consistent', 'fair', 'use', 'please', 'go', 'image', 'description', 'page', 'edit', 'include', 'fair', 'use', 'rationale', 'uploaded', 'fair', 'use', 'medium', 'consider', 'checking', 'specified', 'fair', 'use', 'rationale', 'page', 'find', 'list', "'image", 'page', 'edited', 'clicking', '``', "''", 'contribution', "''", "''", 'link', 'located', 'top', 'wikipedia', 'page', 'logged', 'selecting', '``', "''", 'image', "''", "''", 'dropdown', 'box', 'note', 'fair', 'use', 'image', 'uploaded', '4', 'may', '2006', 'lacking', 'explanation', 'deleted', 'one

### One-hot vectorization and count vectorization

In [0]:
sample_vector = np.zeros(len(cleaned_vocab))

for token in sample_text: 
    try: 
        sample_vector += vocabulary[token]
    except KeyError: 
        continue

In [131]:
sample_vector

array([3., 0., 0., ..., 0., 0., 0.])

Right now we have count vectorization for our text.   
Use this pipeline to create vectors for all of the texts. Save them into np.array. i-th raw in np.array is a vector which represents i-th text from the dataframe.  

In [132]:
matrix_size = 1000
df_sample = df.cleaned.sample(matrix_size)

def vect(size, dataset):
  c = 0
  text_matrix = np.zeros((size, len(cleaned_vocab)))
  for i in dataset:
    comment_matrix = np.zeros(len(cleaned_vocab))
    for token in i:
      try:
        comment_matrix = comment_matrix + vocabulary[token]
      except KeyError:
        continue
    text_matrix[c] = comment_matrix
    c = c + 1
  return text_matrix

vectorized_text = vect(matrix_size, df_sample)
print(vectorized_text)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### The next step is to train any classification model on top of the received vectors and report the quality. 

Please, select any of the proposed pipelines for performing a text classification task. (Binary, multiclass or multilabel).  

The main task to calculate our models performance is to create a training and test sets. When you selected a texts for your task, please, use https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html to have at least two sets - train and test.  

Train examples you will use to train your model on and test examples to evaluate your model - to understand how your model works on the unseen data. 

### Train-test split 

In [147]:
from sklearn.model_selection import train_test_split

df_small = df[:2000]
test_data, train_data = train_test_split(df_small, random_state = 50, test_size = 0.5)


print('-------------------------------------test_data-------------------------------------','\n',test_data,'\n\n')
print('-------------------------------------train_data-------------------------------------','\n',train_data)

-------------------------------------test_data------------------------------------- 
                     id  ...                                            cleaned
838   024859995cf6379e  ...  [thank, experimenting, wikipedia, test, worked...
1492  0400b22cadd183f4  ...  [external, link, 've, added, official, bobby, ...
515   01588a3bc3e2755d  ...  [``, =^_^=, watched, entire, series, agree, en...
1357  03b0df8e2f79ad61  ...  [understood, term, art, literal, enunciation, ...
1573  0439882932f5f3ca  ...  [``, muhammad, view, slavery, article, un-bala...
...                ...  ...                                                ...
1313  0392de34030477e6  ...  [``, agree, actual, usage, ``, '', russia, '',...
109   004a23742282fee4  ...  [welcome, wikipedia, bla, discover, ekopedia, ...
1931  053ceb31fc1a90a8  ...  [comment, password, hacking, really, n't, thin...
1504  040a95741d7c8d91  ...  [non-involved, people, definitely, understand,...
1712  04a67e3c97228690  ...  [``, oops, misun

In [145]:
test_data['cleaned']

493    [go, away, revert, edit, blowdart, 's, user, t...
322    [total, population, 10, million, doe, match, e...
372    [upgraded, human, language, family, wikicolors...
735    [pkk, fighter, past, decade, entering, living,...
680    [procedure, ask, award, michelle, obama, alrea...
                             ...                        
289    ['m, focussing, science, moment, leading, revo...
109    [welcome, wikipedia, bla, discover, ekopedia, ...
907    [``, trying, measure, number, cafe, one, place...
480    [mark, ii, subsec, removed, mark, iix, subsect...
688    [``, past, master, vs., rarity, 'm, creating, ...
Name: cleaned, Length: 500, dtype: object

### TF-IDF score 

#### Please, review again this article or read it if you have not done it before. 

https://medium.com/@paritosh_30025/natural-language-processing-text-data-vectorization-af2520529cf7

#### Implement calculating a tf-idf score for each of the words from your vocabulary. 

The main goal of this taks is to create a dictionary - keys of the dictionary would be tokens and values would be corresponding tf-idf score of the token.

#### Calculate it MANUALLY and compare the received scores for words with the sklearn implementation:  
from sklearn.feature_extraction.text import TfidfTransformer 

#### Tip: 

##### TF = (Number of time the word occurs in the current text) / (Total number of words in the current text)  

##### IDF = (Total number of documents / Number of documents with word t in it)

##### TF-IDF = TF*IDF 

When you calculated a tf-idf score for each of the words in your vocabulary - revectorize the texts.  
Instead of using number of occurences of the i-th word in the i-th cell of the text vector, use it's tf-idf score.   

Revectorize the documents, save vectors into np.array. 

In [148]:
def tfscore(word, text, c): #(Number of time the word occurs in the current text) / (Total number of words in the current text)
  occurs = 0
  words = text.split()
  for i in words:
    if i == word:
      occurs = occurs + 1
    if occurs != 0:
      c = c + 1
  tf = occurs / len(words)
  return tf, c

def tfidf(df, documents): #((Number of time the word occurs ...) / (Total number of words...))*((Total number of documents / Number of documents with word t in it))
  word_row = []
  for word in documents:
    c = 0
    word_column = []
    for comment_text in df['comment_text']:
      probability, c = tfscore(word, comment_text, c)
      word_column.append(probability)
    mult = c/len(documents)
    word_row.append(np.array(word_column) * mult)
  return np.array(word_row)

docs = []
for j in cnt_vocab.most_common(1000):
  docs.append(j[0])
result = tfidf(df_small, docs)
print(result)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.26109091 0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


### Training the model 

As it was said before, select any of the text classification models for the selected task and train the model. 

When the model is trained, you need to evaluate it somehow. 

Read about True positive, False positive, False negative and True negative counts and how to calculate them:   

https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative 

##### Calculate TP, FP, FN and TN on the test set for your model to measure its performance. 


In [149]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix

model = KNeighborsClassifier()
tfidf = TfidfVectorizer(vocabulary = cleaned_vocab)
x = tfidf.fit_transform(train_data.cleaned.str.join(' '))
x_test = tfidf.fit_transform(test_data.cleaned.str.join(' '))
y = train_data.drop(columns = ['id','comment_text','cleaned']).values
y_test = test_data.drop(columns = ['id','comment_text','cleaned']).values
model.fit(x,y)
prediction = model.predict(x_test)

TP = np.sum(np.logical_and(prediction == 1, y_test == 1)) #True positive
FP = np.sum(np.logical_and(prediction == 1, y_test == 0)) #False positive
TN = np.sum(np.logical_and(prediction == 0, y_test == 0)) #True negative
FN = np.sum(np.logical_and(prediction == 0, y_test == 1)) #False negative

print(TP, FP, TN, FN)

5 43 5743 209


#### The next step is to calculate  Precision, Recall, F1 and F2 score 

https://en.wikipedia.org/wiki/Sensitivity_and_specificity

In [151]:
prec = TP/(FP+TP)
rec = TP/(TP+FN)
F1 = (2*TP)/(2*TP + FP + FN)
F2 = (5*prec*rec)/(4*prec + rec)
print(prec, rec, F1, F2)

0.10416666666666667 0.02336448598130841 0.03816793893129771 0.027654867256637166


Calculate these metrics for the vectorization created using count vectorizing and for tf-idf vectorization.  
Compare them. 

### Conclusions and improvements 

For all of the vectorization pipelines we used all of the words, which were available in our dictionary, as experiment try to use the most meaningful words - select them using TF-IDF score. (for example for each text you can select not more than 10 words for vectorization, or less). 

Compare this approach with the first and second ones. Did your model improve? 



### Additionally, visualisations 

For now you have a vector for each word from your vocabulary. 
You have vectors with lenght > 18000, so the dimension of your space is more than 18000 - it's impossible to visualise it in 2d space. 

So try to research and look for algorithms which perform dimensionality reduction. (t-SNE, PCA) 
Try to visualise obtained vectors in a vectorspace, only subset from the vocabulary, don't plot all of the words. (100) 

Probably on this step you will realise how this type of vectorization using these techniques is not the best way to vectorize words. 

Please, analyse the obtained results and explain why visualisation looks like this. 